In [116]:
from sklearn.metrics import classification_report
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.callbacks import TensorBoard
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
from tqdm import tqdm
import pathlib
from google.cloud import storage
import tensorflow_hub as hub
from io import BytesIO
from tensorflow.python.lib.io import file_io

In [117]:
# train_data = pd.read_csv("gs://dataset-gabon/train_val_test_csv_Siebren/train_data_siebren.csv", header=None)
val_data = pd.read_csv("gs://dataset-gabon/train_val_test_csv_Siebren/val_set.csv")
# test_data = pd.read_csv("gs://dataset-gabon/train_val_test_csv_Siebren/test_data_siebren.csv", header=None)
# CLASS_NAMES = train_data[0].unique()

In [118]:
elephant = val_data[val_data["elephant"] == 1]

In [119]:
blank = val_data[val_data["blank"] == 1].sample(frac=0.01)

In [120]:
val_data = val_data.drop(val_data[val_data["elephant"] == 1].index)

In [121]:
val_data = val_data.drop(val_data[val_data["blank"] == 1].index)

In [122]:
other = val_data.sample(frac=0.025)

In [56]:
humans = pd.DataFrame(columns=['class', 'link'])
client = storage.Client()
for blob in tqdm(client.list_blobs('dataset-myanmar-renamed', prefix='SnapshotSerengeti_humans_v2.0')):
    if blob.name[-3:].lower() == "jpg":
        humans = humans.append({"class": "Human", 'link': "gs://dataset-myanmar-renamed/"+str(blob.name)}, ignore_index=True)

47038it [02:53, 271.34it/s]


In [68]:
humans = humans.sample(frac=0.057)
humans = humans.rename(columns = {'class': 'label', 'link': 'file_name'}, inplace = False)

In [124]:
def make_label(data, label):
    data["file_name"] = "gs://justdiggit-serengeti-data-512/decompressed/512_" + data["file_name"]
    data['label'] = label
    data = data[['label', 'file_name']]
    return data

other_data = make_label(other, "Other_animal")
elephant_data = make_label(elephant, "Elephant_African")
blank_data = make_label(blank, "Blank")

In [129]:
serengeti = humans.append(other_data, ignore_index=True)
serengeti = serengeti.append(elephant_data, ignore_index=True)
serengeti = serengeti.append(blank_data, ignore_index=True)
# serengeti = serengeti.reset_index()

In [130]:
serengeti

,label,file_name
0,Human,gs://dataset-myanmar-renamed/SnapshotSerengeti...
1,Human,gs://dataset-myanmar-renamed/SnapshotSerengeti...
2,Human,gs://dataset-myanmar-renamed/SnapshotSerengeti...
3,Human,gs://dataset-myanmar-renamed/SnapshotSerengeti...
4,Human,gs://dataset-myanmar-renamed/SnapshotSerengeti...
...,...,...
10582,Blank,gs://justdiggit-serengeti-data-512/decompresse...
10583,Blank,gs://justdiggit-serengeti-data-512/decompresse...
10584,Blank,gs://justdiggit-serengeti-data-512/decompresse...
10585,Blank,gs://justdiggit-serengeti-data-512/decompresse...


In [131]:
serengeti.to_csv("gs://dataset-gabon/train_val_test_csv_Siebren/serengeti_siebren.csv", index=None, header=None)